In [1]:
import requests
from bs4 import BeautifulSoup
import numpy as np
from lxml import etree
import json, io
import re
from pathlib import Path


In [15]:
def jsonForcast(code):
    try:
        to_unicode = unicode
    except NameError:
        to_unicode = str
    urlBase = 'http://emweb.securities.eastmoney.com/PC_HSF10/ProfitForecast/ProfitForecastAjax?code='
    shOrSz = lambda x: ('sz' + code) if code[0] in {'0', '2', '3'} else 'sh' + code
    code = shOrSz(code)
    saveJsonStr = './data/forcast/%s.json' % code
    saveJson = Path(saveJsonStr)
    if (saveJson.exists() ):  # todo 更新
        with open(saveJsonStr, "r", encoding = "utf-8") as data_file:
            stockJson = json.load(data_file)
    else:
        html = requests.get(urlBase+code)
        stockJson = json.loads(html.text)
        saveJson.touch()
        with io.open(saveJsonStr, 'w', encoding='utf8') as outfile:
            str_ = json.dumps(stockJson,
                      indent=4, sort_keys=True,
                      separators=(',', ': '), ensure_ascii=False)
            outfile.write(to_unicode(str_))
    return stockJson

def zh2float(string):
    if('亿' == string[-1]):
        return float(string[:-1]) * 100000000
    elif('万' == string[-1]):
        return float(string[:-1]) * 10000

def jlrForcast(code):
    stockJson = jsonForcast(code)
    for i in stockJson['Result']['yctj']['data']:
        if (i['rq'].endswith('年预测')):
            if (i['jlr'] == '--'):
                return np.NaN
            jlr = zh2float(i['jlr'].split('(')[0])
#             print('预测： ' + str(jlr) + ' L: ' + str(jlrLast))
            nprg_forcast = round((jlr - jlrLast) / jlrLast * 100, 2)
            break
        jlrLast = zh2float(i['jlr'].split('(')[0])
    return nprg_forcast
    
    

In [27]:
from importlib import reload
import MyStock as ms

reload(ms)

test = ms.MyStock()

In [28]:
test.nprgForcast('300593')

2.04

In [29]:
test.nprgForcast('002614')

33.86

In [13]:
URL = ("http://data.eastmoney.com/bbsj/stock600057/yjbb.html")
html = requests.get(URL)
t = Path('test.html')
with t.open('w') as out:
    out.write(html.text)

In [6]:
from importlib import reload
import MyStock as ms

reload(ms)

<module 'MyStock' from 'D:\\Anaconda_code\\MyStock.py'>

In [7]:
test = ms.MyStock()

test.research()

[Getting data:]#[Getting data:]###

In [8]:
test.stockGDF.head()

,code,name0,mbrg0,nprg0,nav0,targ0,epsg0,seg0,name1,mbrg1,...,eps,rec_report_date,nprg,pe_report_date,peg_report_date,price_report_date,price_now,涨幅,pe,peg
0,300593,新雷能,32.2045,1554.8818,24.7699,22.1790,NaN,24.7699,新雷能,25.6332,...,0.51,2017-04-20,30.5910,77.11,2.52,29.959,20.11,-0.328749,57.57,1.88
1,002614,奥佳华,25.8694,287.5262,4.0064,19.2742,295.4545,3.8546,奥佳华,26.0666,...,0.46,2017-04-25,45.8953,123.98,2.70,16.708,19.12,0.144362,38.26,0.83
2,603579,荣泰健康,25.9892,269.4559,-8.0437,27.7160,NaN,-8.3904,荣泰健康,75.7753,...,3.94,2017-02-27,45.3725,41.56,0.92,59.265,65.44,0.104193,44.33,0.98
3,601179,中国西电,4.5743,232.0633,25.2853,11.1603,159.2593,25.6805,中国西电,6.2670,...,0.22,2017-04-11,25.6954,29.81,1.16,6.697,4.36,-0.348962,25.77,1.00
4,600867,通化东宝,21.4530,200.8781,1.2569,13.2066,150.0000,1.5026,通化东宝,20.5191,...,0.46,2017-04-12,30.4331,47.34,1.56,17.459,22.34,0.279569,44.21,1.45


In [30]:
test.stockGDF['nprg_forcast'] = test.stockGDF.apply(lambda x: test.nprgForcast(x['code']), axis=1)

AttributeError: 'MyStock' object has no attribute 'stockGDF'

In [10]:
test.stockGDF['peg_forcast'] = test.stockGDF.apply(lambda x: np.NaN if np.isnan(x['nprg_forcast']) else x['pe']/x['nprg_forcast'], axis=1)

In [12]:
test.stockGDF[['code','name2', 'peg_forcast', 'nprg_forcast', 'pe_report_date', 'nprg', 'price_now', 'rec_report_date', 'price_report_date', '涨幅']         
              + ['nprg'+str(i) for i in range(5)]][(test.stockGDF.peg_forcast < 0.4) \
                                                                                    & (test.stockGDF.peg_forcast > 0)]

,code,name2,peg_forcast,nprg_forcast,pe_report_date,nprg,price_now,rec_report_date,price_report_date,涨幅,nprg0,nprg1,nprg2,nprg3,nprg4
7,300274,阳光电源,0.360407,63.90,27.25,28.1662,15.92,2017-04-18,9.904,0.607431,148.6635,56.4849,50.4169,28.1662,NaN
10,002019,亿帆医药,0.321407,65.96,26.63,88.8034,20.53,2017-03-31,16.747,0.225891,126.2480,829.1151,52.1242,88.8034,NaN
11,600057,象屿股份,0.341902,55.63,28.20,45.5723,8.61,2017-04-26,9.619,-0.104897,113.0045,32.1064,30.1014,45.5723,NaN
21,300432,富临精工,0.170512,116.59,35.76,31.7331,16.39,2017-04-26,23.149,-0.291978,63.5701,46.3348,27.0005,31.7331,NaN
33,000756,新华制药,0.266063,98.36,53.18,41.5265,12.67,2017-03-15,14.326,-0.115594,45.8721,78.3765,37.5711,41.5265,NaN
34,002078,太阳纸业,0.209540,77.36,16.51,53.3837,11.46,2017-04-25,6.962,0.646079,44.9830,72.0237,38.0570,53.3837,NaN
48,002589,瑞康医药,0.284485,70.90,45.40,177.6553,12.91,2017-03-31,17.790,-0.274311,30.3360,26.0113,33.7655,177.6553,NaN
